### Table of Content
- [`passivating_molecule` into SMILES format](Converting-`passivating_molecule`-into-SMILES-format)
- [`perovskite_composition` into features](`perovskite_composition`-into-features)
- [baseline ML model](baseline-ML-model)

In [2]:
import pandas as pd
import numpy as np
import pubchempy as pcp

In [3]:
df = pd.read_json('data/finetuned_llama_output.json')
data = df.T.sort_index()
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 149 entries, 0 to 149
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   control_pce               71 non-null     object
 1   control_voc               53 non-null     object
 2   treated_pce               140 non-null    object
 3   treated_voc               124 non-null    object
 4   passivating_molecule      143 non-null    object
 5   perovskite_composition    134 non-null    object
 6   electron_transport_layer  118 non-null    object
 7   hole_transport_layer      115 non-null    object
 8   pin_nip_structure         147 non-null    object
 9   stability_tests           149 non-null    object
 10  pin_structure             1 non-null      object
dtypes: object(11)
memory usage: 14.0+ KB


In [4]:
# Function to select columns
def select_data(df):
    # Convert PCE and VOC to numeric
    for col in ['control_pce', 'control_voc', 'treated_pce', 'treated_voc']:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Drop rows where treated_pce or passivating_molecule is missing
    df = df.dropna(subset=['treated_pce', 'passivating_molecule', 'perovskite_composition'])

    return df

data = select_data(data)
data.head()

,control_pce,control_voc,treated_pce,treated_voc,passivating_molecule,perovskite_composition,electron_transport_layer,hole_transport_layer,pin_nip_structure,stability_tests,pin_structure
0,25.7,1.17,26.15,1.18,4-chlorobenzenesulfonate (4Cl-BZS),α-phase FAPbI3,C60,SAMs (self-assembled monolayers),PIN,"[{'test_name': 'ISOS-D-2I', 'temperature': 85,...",NaN
3,24.5,1.20,24.50,1.20,BA2MA2Pb3I10,BA2MA2Pb3I10,SnO2,PTAA,PIN,"[{'test_name': 'ISOS-L-1', 'temperature': None...",NaN
4,NaN,NaN,21.06,1.14,vinylbenzylammonium bromide (VBABr),(FAPbI3)0.95(MAPbBr3)0.05,Spiro-OMeTAD,Spiro-OMeTAD,PIN,"[{'test_name': None, 'temperature': None, 'tim...",NaN
5,NaN,NaN,22.10,1135.00,iso-BAI,FA(MA)PbI3,Spiro-OMeTAD,TPBI,PIN,"[{'test_name': 'ISOS-LT', 'temperature': None,...",NaN
6,NaN,NaN,15.30,1.06,phenylethylammonium,PEA2(CH3NH3)n-1Pb(nI3n+1),TiO2,spiro-OMeTAD,PIN,"[{'test_name': 'ISOS-D', 'temperature': None, ...",NaN


In [5]:
data.isna().sum(axis=0)

control_pce                  59
control_voc                  73
treated_pce                   0
treated_voc                  10
passivating_molecule          0
perovskite_composition        0
electron_transport_layer     19
hole_transport_layer         20
pin_nip_structure             0
stability_tests               0
pin_structure               107
dtype: int64

## `passivating_molecule` into SMILES format

In [7]:
def fetch_smiles_from_name(molecule_name):
    try:
        # Search for the molecule in PubChem by name
        compounds = pcp.get_compounds(molecule_name, 'name')
        if compounds:
            return compounds[0].isomeric_smiles  # Return the first match's SMILES
        else:
            return np.nan
    except Exception as e:
        print(f"Error fetching SMILES for {molecule_name}: {e}")
        return None

In [8]:
data['passivating_molecule']

0                     4-chlorobenzenesulfonate (4Cl-BZS)
3                                           BA2MA2Pb3I10
4                    vinylbenzylammonium bromide (VBABr)
5                                                iso-BAI
6                                    phenylethylammonium
                             ...                        
144                                               GlyHCl
145                                4-fluoroaniline (FAL)
147    4-(3,6-bis(4-(bis(4-methoxyphenyl)amino)phenyl...
148           4-tert-butyl-benzylammonium iodide (tBBAI)
149            4-trifluoromethyl-phenylammonium (CF3-PA)
Name: passivating_molecule, Length: 107, dtype: object

In [9]:
data['passivating_molecule'].apply(fetch_smiles_from_name).isna().sum()

84

### Cleaning Data

In [11]:
import re

def fix_unmatched_brackets(s):
    """
    Fixes unmatched brackets in the given string by adding the correct brackets where necessary.

    :param s: Input string with potential unmatched brackets.
    :return: A corrected string with properly balanced brackets.
    """
    opening = "({["
    closing = ")}]"
    match = {')': '(', '}': '{', ']': '['}
    stack = []

    # Step 1: Identify missing closing brackets
    fixed_s = []
    for char in s:
        if char in opening:
            stack.append(char)
            fixed_s.append(char)
        elif char in closing:
            if stack and stack[-1] == match[char]:
                stack.pop()
                fixed_s.append(char)
            else:
                # Add missing opening bracket before unmatched closing
                fixed_s.insert(0, match[char])
                fixed_s.append(char)
        else:
            fixed_s.append(char)

    # Step 2: Add missing closing brackets at the end
    while stack:
        open_bracket = stack.pop()
        fixed_s.append(closing[opening.index(open_bracket)])

    return "".join(fixed_s)


def get_chemical_names(chemical_list):
    cleaned_list = []
    for name in chemical_list:
        # Remove text inside parentheses at the end if it's extra information (abbreviations)
        name = re.sub(r"\s*\([^)]*\)$", "", name).strip()

        cleaned_list.append(name)

    return cleaned_list

In [12]:
# Example input list
lst = data['passivating_molecule']

# Cleaning the list
cleaned_list = lst.apply(lambda x: fix_unmatched_brackets(x))
cleaned_list = get_chemical_names(cleaned_list)

# Output result
data['passivating_molecule_cleaned'] = cleaned_list

In [ ]:
data['passivating_molecule_SMILES'] = data['passivating_molecule_cleaned'].apply(fetch_smiles_from_name)

## `perovskite_composition` into features

In [ ]:
data['perovskite_composition']#.apply(lambda x: x.split(' ')))

In [ ]:
import chemparse

In [ ]:
# Example formula
formula = "(FAPbI3)0.95(MAPbBr3)0.05"

# Parse formula
result = chemparse.parse_formula(formula)
print(result)  # Output: {'FA': 0.95, 'Pb': 1.0, 'I': 2.85, 'MA': 0.05, 'Br': 0.15}

In [ ]:
from sympy import sympify

# Example formula with weights
formula = "0.95 * (FA + Pb + 3*I) + 0.05 * (MA + Pb + 3*Br)"

# Evaluate
expanded_formula = sympify(formula)
print(expanded_formula)

## baseline ML model